In [2]:
import pandas as pd


In [83]:
Weather_full = pd.read_csv("./Wetterdaten_Basel_stündlich22_24.csv", skiprows=9)

In [84]:
Weather_full.head()


,timestamp,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
0,20220101T0000,9.060529,0.0,7.729527,152.24146
1,20220101T0100,8.900529,0.0,8.287822,145.61966
2,20220101T0200,8.740529,0.0,8.145870,135.00000
3,20220101T0300,8.290529,0.0,7.704336,127.40535
4,20220101T0400,7.980528,0.0,7.729527,117.75854


In [85]:
# Rename the column
Weather_full.rename(columns={'timestamp': 'Datum'}, inplace=True)

# Change the format of the 'Datum' column
Weather_full['Datum'] = pd.to_datetime(Weather_full['Datum'], format='%Y%m%dT%H%M', utc=True)

# Sort by 'Datum' and set it as the index
Weather_full.sort_values(by='Datum', inplace=True)
Weather_full.set_index('Datum', inplace=True)

# Display the first few rows to verify the changes
Weather_full.head()

,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
Datum,,,,
2022-01-01 00:00:00+00:00,9.060529,0.0,7.729527,152.24146
2022-01-01 01:00:00+00:00,8.900529,0.0,8.287822,145.61966
2022-01-01 02:00:00+00:00,8.740529,0.0,8.145870,135.00000
2022-01-01 03:00:00+00:00,8.290529,0.0,7.704336,127.40535
2022-01-01 04:00:00+00:00,7.980528,0.0,7.729527,117.75854


In [88]:
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

target_columns = ["Basel Temperature [2 m elevation corrected]", "Basel Precipitation Total", "Basel Wind Speed [10 m]", "Basel Wind Direction [10 m]"]

for col in target_columns:
    Weather_full[col] = scaler.fit_transform(Weather_full[col].values.reshape(-1, 1))


means = Weather_full[target_columns].mean()
stds = Weather_full[target_columns].std()

print(f"Means: {means}")

print(f"Stds: {stds}")

Weather_full.head()

Means: Basel Temperature [2 m elevation corrected]   -4.487875e-18
Basel Precipitation Total                     -2.692725e-17
Basel Wind Speed [10 m]                       -1.346362e-17
Basel Wind Direction [10 m]                    2.692725e-17
dtype: float64
Stds: Basel Temperature [2 m elevation corrected]    1.00002
Basel Precipitation Total                      1.00002
Basel Wind Speed [10 m]                        1.00002
Basel Wind Direction [10 m]                    1.00002
dtype: float64


,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
Datum,,,,
2022-01-01 00:00:00+00:00,-0.612920,-0.179497,-0.470399,-0.507739
2022-01-01 01:00:00+00:00,-0.632843,-0.179497,-0.400364,-0.580393
2022-01-01 02:00:00+00:00,-0.652767,-0.179497,-0.418171,-0.696911
2022-01-01 03:00:00+00:00,-0.708802,-0.179497,-0.473559,-0.780239
2022-01-01 04:00:00+00:00,-0.747404,-0.179497,-0.470399,-0.886083


In [89]:
#hopefully useless
#checking for missing values
Weather_full.reset_index(inplace=True)

# Create a date range from the minimum to the maximum date in the 'Datum' column
date_range = pd.date_range(start=Weather_full['Datum'].min(), end=Weather_full['Datum'].max(), freq='D')

# Create a DataFrame with all the dates in the range
all_dates = pd.DataFrame({'Datum': date_range})

# Merge this new DataFrame with the original to find missing dates
missing_dates = pd.merge(all_dates, Weather_full, on='Datum', how='left', indicator=True)

# Rows that are missing in the original DataFrame will have '_merge' == 'left_only'
missing_rows = missing_dates[missing_dates['_merge'] == 'left_only']

# Output the missing rows (if any)
if not missing_rows.empty:
    print(f"Missing rows for the following dates: {missing_rows['Datum'].tolist()}")
else:
    print("No rows are missing.")

No rows are missing.


In [66]:
Weather_full.to_csv("./Cleaned/Wetterdaten_Basel_cleanded22_24.csv")

In [124]:
Gas_full = pd.read_csv("./Gasverbrauch_im_Versorgungsgebiet_der_IWB.csv", sep=';')

In [125]:
Gas_full.head()

,Start der Messung,Gasverbrauch,Datum,Zeit,Jahr,Monat,Tag,Wochentag,Tag des Jahres,Quartal,Woche des Jahres
0,2023-10-18T23:00:00+00:00,185054.4299,2023-10-19,01:00,2023,10,19,3,292,4,42
1,2023-10-19T00:00:00+00:00,184888.9114,2023-10-19,02:00,2023,10,19,3,292,4,42
2,2023-10-19T01:00:00+00:00,196871.6645,2023-10-19,03:00,2023,10,19,3,292,4,42
3,2023-10-19T04:00:00+00:00,322512.8374,2023-10-19,06:00,2023,10,19,3,292,4,42
4,2023-10-19T05:00:00+00:00,348277.5490,2023-10-19,07:00,2023,10,19,3,292,4,42


In [126]:
Gas_full.sort_values(by="Start der Messung", inplace=True)
Gas_full.head()

Gas_full.drop(columns=["Datum", "Zeit", "Jahr", "Monat", "Tag", "Wochentag", "Tag des Jahres", "Quartal", "Woche des Jahres"], inplace=True)

Gas_full['Start der Messung'] = pd.to_datetime(Gas_full['Start der Messung'], utc=True)

Gas_full.rename(columns={'Start der Messung': 'Datum'}, inplace=True)
#Gas_full.set_index('Datum', inplace=True)
Gas_full = Gas_full[Gas_full['Datum'] >= '2022-01-01']
#Gas_full.index = Gas_full.index.str.replace('T', ' ')

In [127]:


#hopefully useless
#checking for missing values
Gas_full.reset_index(inplace=True)

# Create a date range from the minimum to the maximum date in the 'Datum' column
date_range = pd.date_range(start=Gas_full['Datum'].min(), end=Gas_full['Datum'].max(), freq='D')

# Create a DataFrame with all the dates in the range
all_dates = pd.DataFrame({'Datum': date_range})

# Merge this new DataFrame with the original to find missing dates
missing_dates = pd.merge(all_dates, Gas_full, on='Datum', how='left', indicator=True)

# Rows that are missing in the original DataFrame will have '_merge' == 'left_only'
missing_rows = missing_dates[missing_dates['_merge'] == 'left_only']

# Output the missing rows (if any)
if not missing_rows.empty:
    print(f"Missing rows for the following dates: {missing_rows['Datum'].tolist()}")
else:
    print("No rows are missing.")

Missing rows for the following dates: [Timestamp('2022-10-30 00:00:00+0000', tz='UTC'), Timestamp('2023-10-29 00:00:00+0000', tz='UTC'), Timestamp('2024-10-27 00:00:00+0000', tz='UTC')]


In [129]:
Gas_full.index = Gas_full['Datum']

# Check for duplicates in the index
print(Gas_full.index.duplicated().sum())  # Number of duplicates
print(Gas_full.index[Gas_full.index.duplicated()])  # Which labels are duplicates

duplicates = Gas_full.index.duplicated(keep=False)
print(Gas_full[duplicates])



3
DatetimeIndex(['2022-03-27 01:00:00+00:00', '2023-03-26 01:00:00+00:00',
               '2024-03-31 01:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Datum', freq=None)
                           index                     Datum  Gasverbrauch
Datum                                                                   
2022-03-27 01:00:00+00:00  23891 2022-03-27 01:00:00+00:00   251581.2461
2022-03-27 01:00:00+00:00   3201 2022-03-27 01:00:00+00:00   346278.7600
2023-03-26 01:00:00+00:00  16823 2023-03-26 01:00:00+00:00   236378.3897
2023-03-26 01:00:00+00:00   6553 2023-03-26 01:00:00+00:00   254217.8493
2024-03-31 01:00:00+00:00   4705 2024-03-31 01:00:00+00:00   220161.4192
2024-03-31 01:00:00+00:00  26237 2024-03-31 01:00:00+00:00   244597.7287


In [121]:
import pandas as pd

# Define the specific dates for which you want to apply the rolling mean
specific_dates = [
    pd.Timestamp('2022-10-30 00:00:00+0000', tz='UTC'),
    pd.Timestamp('2023-10-29 00:00:00+0000', tz='UTC'),
    pd.Timestamp('2024-10-27 00:00:00+0000', tz='UTC')
]

# Ensure 'Datum' is in datetime format (if not already)
#Gas_full.index = pd.to_datetime(Gas_full.index)

# Create a mask to filter the specific dates
mask = Gas_full.index.isin(specific_dates)

# Apply the rolling mean to the specified dates only
Gas_full.loc[mask, 'Gasverbrauch'] = Gas_full.loc[mask, 'Gasverbrauch'].fillna(
    Gas_full['Gasverbrauch'].rolling(window=3, min_periods=1).mean()
)

# Output the resulting DataFrame
print(Gas_full.loc[specific_dates])


ValueError: cannot reindex on an axis with duplicate labels

In [106]:
#standardize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Reshape the column to a 2D array
Gas_full["Gasverbrauch_scaled"] = scaler.fit_transform(Gas_full["Gasverbrauch"].values.reshape(-1, 1))

# Verify the mean and standard deviation
Gas_full["Gasverbrauch_scaled"].mean(), Gas_full["Gasverbrauch_scaled"].std()


(np.float64(-5.4991658917662245e-17), np.float64(1.000019743921342))

In [ ]:
Gas_full.drop(columns=["Gasverbrauch"], inplace=True)

print(Gas_full)


Gas_full.to_csv("./Cleaned/Gasverbrauch_cleaned.csv")